In [7]:
#9.1.2 파이썬 표준 웹 라이브러리
import urllib.request as ur
url = 'http://quotesondesign.com/wp-json/posts'
conn = ur.urlopen(url) # 이 url은 포춘쿠키와 유사한 임의의 텍스트 문장을 반환
print(conn) 

data = conn.read() #read() 메서드는 웹페이지로부터 데이터를 읽어온다 
print(data)
# -> 이 코드로 1)원격 인용구 서버에 TCP/IP 커넥션을 열었고, 2)HTTP요청을 만들엇고, 3)HTTP응답을 받았다.
# ID, title, content(페이지 포춘 데이터) 이외의 데이터들이 포함되어있다

print(conn.status) # HTTP 상태코드 -> 200 : 성공적으로 처리됨

#웹서버는 원하는 포맷으로 데이터를 전송할 수 있다.
#일반적으로는 HTML(or CSS or javascript)를 전송 but 포춘쿠키에 대한 예제는 json포맷
#데이터 포맷은 telnet에서 본 듯이 HTTP헤더의 Content-Type 값에 있다.

print(conn.getheader('Content-Type'))
#application/json 문자열은 MIME(Multipurpose Internet Mail Extensions) 타입이다.
#MIME타입은 google.com 예제에서 본 text/html이다.


#파이썬 라이브러리는 모든 HTTP 응답 헤더를 파싱하여 딕셔너리로 제공
for key, value in conn.getheaders():
    print(key, value)


b'[{"ID":2450,"title":"Paula Scher","content":"<p>Less is more and more is more. It\\u2019s the middle that\\u2019s not a good place.<\\/p>\\n","link":"https:\\/\\/quotesondesign.com\\/paula-scher-4\\/"}]'
200
application/json; charset=UTF-8
Server nginx
Date Sat, 12 Aug 2017 07:58:25 GMT
Content-Type application/json; charset=UTF-8
Content-Length 194
Connection close
X-Powered-By PHP/5.4.13
X-Content-Type-Options nosniff
Link </wp-json/posts?page=2>; rel="next", <https://quotesondesign.com/wp-json/posts/2450>; rel="item"; title="Paula Scher"
X-WP-Total 1060
X-WP-TotalPages 1060
Last-Modified Fri, 28 Apr 2017 17:44:57 GMT
X-Powered-By PleskLin


In [ ]:
#HTTP 응답코드 정리
# 1xx (조건부 응답): 서버는 요청을 받았지만, 클라이언트에 대한 몇가지 추가 정보가 필요 
# 2xx (성공): 성공적으로 처리, 200 이외의 모든 성공 코드는 추가사항을 전달한다
# 3xx (리다이렉션): 리소스가 이전되어 클라이언트에 새로운 URL을 응답해준다
# 4xx (클라이언트 에러): 자주 발생하는 404(찾을수없음)는 클라이언트 측에 문제가 있음을 나타냄.
# 5xx (서버 에러): 500은 서버에러, 웹서버와 백엔드 애플리케이션 서버가 연결되어 있지 않다면 502(불량게이트웨이)

In [10]:
#9.1.3 표준 라이브러리를 넘서어 : Requests
#$ pip install requests ->requests 패키지의 최신버전 설치

#requests 모듈을 사용하면 웹 클라이언트 개발이 더쉬워짐
import requests
url = 'http://quotesondesign.com/wp-json/posts'
resp = requests.get(url)
resp

print(resp.text) #urllib.request.urlopen 모듈을 사용한 것과 차이는 없지만, 코드가 더 간편하다


[{"ID":2450,"title":"Paula Scher","content":"<p>Less is more and more is more. It\u2019s the middle that\u2019s not a good place.<\/p>\n","link":"https:\/\/quotesondesign.com\/paula-scher-4\/"}]


In [ ]:
#9.2 웹서버
#파이썬은 웹서버와 서버 사이드 프로그램을 작성하는 데 뛰어난 언어
#다양한 프레임워크 존재(bottle, flask 등등)

#웹 프레임워크는 웹사이트를 구축할 수 있는 기능 제공
#-> 단순한 웹(HTTP) 서버 보다 더 많은 작업 수행 -> 라우팅(서버함수의 URL), 템플릿(HTML을 동적으로 생성), 디버깅


In [11]:
#9.2.1 간단한 파이썬 웹서버
#웹서버 실행코드
# python -m http.server
# 결과 :Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
# 0.0.0.0 : 모든 TCP 주소를 의미 
# 웹클라이언트는 서버가 어떤 주소를 가졌든 상관없이 그곳에 접근할 수 있다.

#웹 브라우저에 http://localhost:8000 입력하면
#터미널에 127.0.0.1 - - [12/Aug/2017 19:39:58] "GET / HTTP/1.1" 200 - 출력

#localhost와 127.0.0.1 은 로컬 컴퓨터에 대한 TCP 동의어다
#그래서 인터넷 연결과 상관없이 명령이 실행

#127.0.0.1 - - [12/Aug/2017 19:39:58] "GET / HTTP/1.1" 200 - 해석
 #1) 127.0.0.1 : 클라이언트 IP주소
 #2) 첫번째 '-' : 원격사용자 이름(발견된 경우)
 #3) 두번째 '-' : 로그인 사용자 이름(요구한 경우)
 #4) [12/Aug/2017 19:39:58] : 접근한 날짜,시간
 #5) "GET / HTTP/1.1" : 웹서버로 보내는 명령 -> HTTP 메서드(GET), 리소스요청(/), HTTP버전(HTTP/1.1)
 #6) 200 : 웹서버로부터 반환된 HTTP 상태코드

#localhost:8000 에 있는 리스트 아무거나 누르고 터미널 확인
#서버는 요청에 대한 로그를 기록 -> 127.0.0.1 - - [12/Aug/2017 19:56:08] "GET /Videos/ HTTP/1.1" 200 - (videos 클릭시 출력)

#python -m http.server 9999 : 기본 포트는 8000이지만 다른 포트 번호를 지정할 수 있다.

#이 기본 서버를 실제 웹사이트에서 사용하면 안된다, 
#아파치, 엔진엑스 같은 전통적인 웹서버는 정적 파일을 더 빠르게 제공
#간단한 기본 서버는 대규모의 서버들이 매개변수를 받아서 동적인 콘텐츠를 처리하는 일을 수행할 수 없다.


SyntaxError: invalid syntax (<ipython-input-11-4baf7c42eadf>, line 3)

In [ ]:
#9.2.2 웹서버 게이트웨이 인터페이스
#동적으로 프로그램을 실행하는 웹서버가 필요하다

#공용 게이트웨이 인터페이스(CGI) -웹 초기 시절
 #클라이언트를 위해 웹서버가 외부 프로그램을 실행 -> 그 결과를 반환 하도록 설계
 #CGI는 클라이언트에서 받은 입력 인자를 서버를 통해 처리하여 외부 프로그램으로 전달
 #->프로그램은 각 클라이언트의 접근을 위해 처음부터 다시 시작 => 이러한 접근 방식은 확장성을 떨어뜨림 -> 작은 프로그램에도 상당 시간 소요

#-> 이런 시동 지연을 피하기 위해 웹서버에 '인터프리터'를 두기 시작
#아파치는 mod_php모듈 내에서는 PHP, mod_perl 모듈 내에서는 펄, mod_python 모듈 내에서는 파이썬 실행
#이러한 동적언어의 코드는 외부프로그램이 아닌 장기적으로 작동하는 아파치 프로세스 내에서 실행

#-> 또 다른 방법은 별도의 장기적으로 작동하는 프로그램 내에서 동적언어를 실행하고, 웹서버와 통신(FastCGI, SCGI)

#파이썬 웹 개발은 파이썬 웹 애플리케이션과 웹서버 간의 범용적인 API인 웹 서버 게이트웨이 인터페이스(WSGI)의 정의서 부터 시작


In [ ]:
#프레임워크
#웹서버는 HTTP와 WSGI의 세부사항을 처리
#반면, 프레임워크를 사용한다면 파이썬코드를 작성하여 강력한 웹사이트를 만들 수 있다.

#웹 프레임워크는 최소한 클라이언트의 요청과 서버의 응답을 처리 
#웹프레임워크가 제공하는 기능
 #1)라우트 :URL을 해석하여 해당 서버의 파일이나 파이썬 코드를 찾아준다
 #2)템플릿 :서버사이드의 데이터를 HTML페이지에 병합
 #3)인증 및 권한 : 사용자 이름과 비밀번호, 퍼미션(허가)를 처리
 #4)세션 : 웹사이트에 방문하는 동안 사용자의 임시 데이터를 유지


In [ ]:
#9.2.4 Bottle
#설치 : pip install bottle
